In [17]:
# Run this cell: 
# The lines below will instruct jupyter to reload imported modules before 
# executing code cells. This enables you to quickly iterate and test revisions
# to your code without having to restart the kernel and reload all of your 
# modules each time you make a code change in a separate python file.

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [123]:
import os
# Change path to project root
if os.getcwd().endswith("notebooks"):
    os.chdir(os.path.dirname(os.getcwd()))
print(os.getcwd())

/Users/shloknatarajan/stanford/research/daneshjou/AutoGKB


In [124]:
from src.annotation_extraction.simple_inference import SimpleLLM, PromptGenerator
from src.annotation_extraction.models import ArticleParser
from typing import List

In [125]:
article_title = ArticleParser(pmcid="PMC11730665").parse().title
article_text = ArticleParser(pmcid="PMC11730665").parse().article_text

2025-06-13 13:40:33.257 | INFO     | src.annotation_extraction.models:__init__:13 - Getting article text from PMCID: PMC11730665
2025-06-13 13:40:33.261 | INFO     | src.annotation_extraction.models:remove_references_section:121 - Removed References section from article text
2025-06-13 13:40:33.262 | INFO     | src.annotation_extraction.models:__init__:13 - Getting article text from PMCID: PMC11730665
2025-06-13 13:40:33.264 | INFO     | src.annotation_extraction.models:remove_references_section:121 - Removed References section from article text


In [126]:
DRUG_SUMMARY = """
You are an expert pharmacogenomics researcher reading and extracting annotations from the following article

\n\n{article_text}\n\n

From this article:
First, briefly summarize the methods and results of the study
Next, create a list of extract all variants that have a studied described effect on drug response.
For each variant <-> drug relationship in the list output the following:
Pair: The (variant, drug(s)) pair
Variant: The Variant / Haplotypes (ex. rs2909451, CYP2C19*1, CYP2C19*2, *1/*18, etc.)
Gene: The gene group of the variant (ex. DPP4, CYP2C19, KCNJ11, etc.)
Allele: Specific allele or genotype if different from variant (ex. TT, *1/*18, del/del, etc.)
Relationship Description: Describe the drug(s) in this relationship
Variant Effect: Describe the outcome/effect found from the variant (drug efficacy, metabolism, toxicity, dosage, etc.)
Statistical Analysis: Describe the statistical analysis used and the reported p-values
Population Info: Describe the population of the study participants for this variant
Notes: Describe any other useful information included on this variant to understanding the study results.

These 9 attributes should be separately repeated for every distinct (variant, drug(s)) relationship. For every attribute,
include the information as well as a quote from the article the information was concluded from.
"""

In [158]:
VARIANT_LIST_PROMPT = """
You are an expert pharmacogenomics researcher reading and extracting annotations from the following article:

{article_text}

From this article, note down ALL discussed variants/haplotypes (ex. rs113993960, CYP1A1*1, etc.) that are part of the article's
study/analysis. Include information on the gene group and allele (if present). Your output format should be a list of the variants with the following attributes:
Variant: The Variant / Haplotypes (ex. rs2909451, CYP2C19*1, CYP2C19*2, *1/*18, etc.)
Gene: The gene group of the variant (ex. DPP4, CYP2C19, KCNJ11, etc.)
Allele: Specific allele or genotype if different from variant (ex. TT, *1/*18, del/del, etc.)
"""

In [159]:
from dotenv import load_dotenv
load_dotenv()
from src.annotation_extraction.simple_inference import VariantList, Variant
import json
from loguru import logger

In [160]:
def save_output(prompt, output, filename):
    # save prompt and output to file
    with open(f"test_outputs/{filename}.txt", "w") as f:
        f.write("Prompt:\n")
        f.write(prompt)
        f.write("\nOutput:\n")
        f.write(output)
    logger.info(f"Saved output to {filename}.txt")

In [168]:
from src.annotation_extraction.simple_inference import Variant, VariantList

In [169]:
model = SimpleLLM()
article_text = ArticleParser(pmcid="PMC11730665").get_article_text()
prompt_generator = PromptGenerator(VARIANT_LIST_PROMPT, {"article_text": article_text})
prompt = prompt_generator.get_prompt()
output = model.generate(prompt, response_format=VariantList)

2025-06-13 14:04:22.642 | INFO     | src.annotation_extraction.models:__init__:13 - Getting article text from PMCID: PMC11730665
2025-06-13 14:04:22.645 | INFO     | src.annotation_extraction.models:remove_references_section:121 - Removed References section from article text


In [173]:
from src.annotation_extraction.components import extract_variants_list

ImportError: cannot import name 'VARIANT_LIST_PROMPT' from 'src.annotation_extraction.prompts' (/Users/shloknatarajan/stanford/research/daneshjou/AutoGKB/src/annotation_extraction/prompts.py)

In [171]:
x = json.loads(output)['variant_list']
[i['variant_id'] for i in x]

['rs2909451',
 'rs4664443',
 'rs3765467',
 'rs2285676',
 'rs163184',
 'rs7754840',
 'rs756992',
 'rs1799853',
 'rs1057910']

In [101]:
# save summary to file
save_output(prompt_generator.get_prompt_template(), output, "variant_list_2")

2025-06-12 14:57:23.200 | INFO     | __main__:save_output:8 - Saved output to variant_list_2.txt
